Для начала импортируем необходимые библиотеки. В данной работе я буду использовать алгоритм CatBoostClassifier

In [50]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import metrics
import matplotlib.pyplot as plt
import seaborn as sns
from catboost import CatBoostClassifier
from catboost import Pool
from sklearn.preprocessing import LabelEncoder
import re

Загружаем тренировочный датасет

In [51]:
df_train = pd.read_csv("train.csv")

df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [52]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


Работа с пропусками

Заполним пропуски в столбце Age медианным значением.
Также заполним пропуски в столбце Embarked модой.

Столбец Cabin в процессе обучения использовать не будем, так как там слишком большое количество пропусков и я не вижу возможности извлечь оттуда полезные данные

In [53]:
df_train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [54]:
df_train['Age'].fillna(28, inplace=True)
df_train['Embarked'].fillna('S', inplace=True)

df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          891 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     891 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


Feature engineering

Для начала извлечем информацию из столбца Name. Можно заметить, что в каждом имени есть титул: Mr, Mrs, Dr и т.д. С помощью регулярного выражения вычленим все титулы и логически объединим их в несколько категорий

In [55]:
def substrings(gen_string):
	matches = set()
	regex_pattern = r'[A-Za-z]+\.'
	for string in gen_string:
		matches.add(' '.join(re.findall(regex_pattern, string)).replace('.', ''))
	return matches

titles_train = list(substrings(df_train['Name']))
titles_train

['Don',
 'Countess',
 'Jonkheer',
 'Dr',
 'Col',
 'Major',
 'Capt',
 'Rev',
 'Mme',
 'Miss',
 'Ms',
 'Mrs',
 'Sir',
 'Mlle',
 'Master',
 'Mr',
 'Lady',
 'Mrs L']

In [56]:
def substrings_in_string(big_string, substrings):
    for substring in substrings:
        if big_string.find(substring) != -1:
            return substring
    return np.nan

df_train['Title']=df_train['Name'].map(lambda x: substrings_in_string(x, titles_train))
df_train['Title'].value_counts()

Title
Mr          506
Miss        180
Mrs         125
Master       40
Dr           11
Col          10
Rev           6
Sir           3
Don           2
Major         2
Mme           1
Ms            1
Mlle          1
Capt          1
Countess      1
Jonkheer      1
Name: count, dtype: int64

Изучив значения титулов в интернете, я решил выделить 4 основных категории:

1. Mr - сюда я отнес титулы Mr, Dr(в случае, если пассажир - мужчина), а также другие малочисленные группы титулов, относящихся к мужчинам.

2. Miss - так как наблюдается явное разделение женщин на 2 основные группы - Miss и Mrs, я решил оставить их, распределив оставшиеся малочисленные женские титулы по этим двум категориям. Сюда я отнёс титул Ms (Miss), а также, так как титул Miss имеют незамужние женщины, сюда же я отнёс женщин с титулом Mlle(Mademoiselle), который обычно употребляют в отношении молодых девушек.

3. Mrs - этот титул употребляют в отношении замужнищ женщин, поэтому сюда я, помимо собственно титула Mrs, субъективно, отнёс титул Countess(Графиня), Mme(Мадам) и Lady(Леди)

4. Master - В связи с тем, что данный титул имеют большое количество человек, я решил выделить его в отдельную категорию

In [57]:
def replace_titles(x):
    title=x['Title']
    if title in ['Don', 'Major', 'Capt', 'Jonkheer', 'Rev', 'Col', 'Sir']:
        return 'Mr'
    elif title in ['Countess', 'Mme', 'Lady']:
        return 'Mrs'
    elif title in ['Mlle', 'Ms']:
        return 'Miss'
    elif title =='Dr':
        if x['Sex']=='Male':
            return 'Mr'
        else:
            return 'Mrs'
    else:
        return title
    
df_train['Title']=df_train.apply(replace_titles, axis=1)
df_train['Title'].value_counts()

Title
Mr        531
Miss      182
Mrs       138
Master     40
Name: count, dtype: int64

Объеденим столбцы SibSp(Siblings + Spouses, братья и сестры + супруги) и Parch(Parents + Children, Родители + Дети) в один столбец Family_Size, так как, думаю, влияние этих групп родственников на целевую переменную равнозначно

In [58]:
df_train['Family_Size']=df_train['SibSp']+df_train['Parch']

Для выявления закономерностей с целью создания дополнительных признаков построим матрицу корреляции

Для этого сначала удалим столбцы, которые не будут использоваться в обучении, и с помощью LabelEncoder закодируем оставшиеся категориальные переменные

Удалим столбцы PassengerId, Name, Cabin и Ticket. Из столбца Name я извлек нужную информацию, в столбце Cabin слишком много пропусков, которые невозможно адекватно заполнить, а из столбца Ticket я не вижу возможности извлечь полезную информацию 

In [59]:
df_train.drop(columns=['Name', 'Cabin', 'Ticket'], axis=1, inplace=True)
df_train.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title,Family_Size
0,1,0,3,male,22.0,1,0,7.2500,S,Mr,1
1,2,1,1,female,38.0,1,0,71.2833,C,Mrs,1
2,3,1,3,female,26.0,0,0,7.9250,S,Miss,0
3,4,1,1,female,35.0,1,0,53.1000,S,Mrs,1
4,5,0,3,male,35.0,0,0,8.0500,S,Mr,0


Разделяем данные на признаки для обучения и целевую переменную

In [60]:
X = df_train.drop(columns=['PassengerId', 'Survived'], axis=1)
y = df_train['Survived']

In [61]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [62]:
# Проверить, как составляют пул
grid_pool = Pool(X, label=y,
                  cat_features=['Sex', 'Embarked', 'Title'])

In [63]:
model_1_pool = Pool(X_train, label=y_train,
                  cat_features=['Sex', 'Embarked', 'Title'])

Обучаем модель в первом приближении

In [64]:
model_1 = CatBoostClassifier(learning_rate=0.1, iterations=200, random_seed=42)
model_1.fit(model_1_pool,
            eval_set=(X_val, y_val),
            verbose=False,
            plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [65]:
metrics.roc_auc_score(y_val, model_1.predict(X_val))

0.802960102960103

С помощью встроенной функции grid_search подбираем глобальные параметры и проводим кросс-валидацию

In [66]:
model_2 = CatBoostClassifier(eval_metric='AUC:hints=skip_train~false')

grid_params = {'learning_rate': [0.01, 0.1, 0.3, 0.5],
               'iterations': [50, 100, 200, 500]}

grid_search_result = model_2.grid_search(grid_params,
                                         grid_pool,
                                         verbose=False
                                         )

0:	learn: 0.8736877	test: 0.8647773	best: 0.8647773 (0)	total: 55.4ms	remaining: 2.71s
1:	learn: 0.8716627	test: 0.8511471	best: 0.8647773 (0)	total: 99.8ms	remaining: 2.39s
2:	learn: 0.8753890	test: 0.8439271	best: 0.8647773 (0)	total: 145ms	remaining: 2.27s
3:	learn: 0.8768704	test: 0.8361673	best: 0.8647773 (0)	total: 202ms	remaining: 2.32s
4:	learn: 0.8739076	test: 0.8423077	best: 0.8647773 (0)	total: 256ms	remaining: 2.31s
5:	learn: 0.8713681	test: 0.8332659	best: 0.8647773 (0)	total: 288ms	remaining: 2.11s
6:	learn: 0.8741483	test: 0.8381916	best: 0.8647773 (0)	total: 346ms	remaining: 2.13s
7:	learn: 0.8726918	test: 0.8382591	best: 0.8647773 (0)	total: 390ms	remaining: 2.04s
8:	learn: 0.8780904	test: 0.8404184	best: 0.8647773 (0)	total: 458ms	remaining: 2.09s
9:	learn: 0.8804930	test: 0.8436572	best: 0.8647773 (0)	total: 523ms	remaining: 2.09s
10:	learn: 0.8813602	test: 0.8437922	best: 0.8647773 (0)	total: 582ms	remaining: 2.06s
11:	learn: 0.8838002	test: 0.8450067	best: 0.864777

In [67]:
grid_search_result['params']

{'iterations': 50, 'learning_rate': 0.3}

Загружаем тестовый датасет и подготавливаем его для проверки качества модели

In [68]:
df_test = pd.read_csv("test.csv")
df_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [69]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.1+ KB


In [70]:
df_test.describe()

,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,418.000000,418.000000,332.000000,418.000000,418.000000,417.000000
mean,1100.500000,2.265550,30.272590,0.447368,0.392344,35.627188
std,120.810458,0.841838,14.181209,0.896760,0.981429,55.907576
min,892.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,996.250000,1.000000,21.000000,0.000000,0.000000,7.895800
50%,1100.500000,3.000000,27.000000,0.000000,0.000000,14.454200
75%,1204.750000,3.000000,39.000000,1.000000,0.000000,31.500000
max,1309.000000,3.000000,76.000000,8.000000,9.000000,512.329200


In [71]:
df_test['Age'].fillna(27, inplace=True)
df_test['Fare'].fillna(14.454200, inplace=True)

Обращаюсь к функции substrings, чтобы вычленить титулы в тестовом датасете найти титулы, которых не было в тренировочном датасете, если такие есть

In [72]:
titles_test = list(substrings(df_test['Name']))
[item for item in titles_test if item not in set(titles_train)]

['Dona']

In [73]:
df_test['Title']=df_test['Name'].map(lambda x: substrings_in_string(x, titles_test))
df_test['Title'].value_counts()

Title
Mr        236
Miss       77
Mrs        70
Master     20
Dr          6
Col         5
Rev         2
Ms          1
Dona        1
Name: count, dtype: int64

Согласно найденной информации, Dona - титул, который дают женам дворян, следовательно, я отнесу его в группу титулов Mrs

In [74]:
def replace_titles_test(x):
    title=x['Title']
    if title in ['Don', 'Major', 'Capt', 'Jonkheer', 'Rev', 'Col', 'Sir']:
        return 'Mr'
    elif title in ['Countess', 'Mme', 'Lady', 'Dona']:
        return 'Mrs'
    elif title in ['Mlle', 'Ms']:
        return 'Miss'
    elif title =='Dr':
        if x['Sex']=='Male':
            return 'Mr'
        else:
            return 'Mrs'
    else:
        return title
    
df_test['Title']=df_test.apply(replace_titles_test, axis=1)
df_test['Title'].value_counts()

Title
Mr        243
Miss       78
Mrs        77
Master     20
Name: count, dtype: int64

In [75]:
df_test['Family_Size']=df_test['SibSp']+df_test['Parch']

In [76]:
df_test.drop(columns=['Name', 'Cabin', 'Ticket'], axis=1, inplace=True)
df_test.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title,Family_Size
0,892,3,male,34.5,0,0,7.8292,Q,Mr,0
1,893,3,female,47.0,1,0,7.0000,S,Mrs,1
2,894,2,male,62.0,0,0,9.6875,Q,Mr,0
3,895,3,male,27.0,0,0,8.6625,S,Mr,0
4,896,3,female,22.0,1,1,12.2875,S,Mrs,2


Делаем предсказание и отправляем его на Kaggle.com

In [77]:
df_test['Survived'] = model_2.predict(df_test.drop(columns='PassengerId', axis=1))
df_test.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title,Family_Size,Survived
0,892,3,male,34.5,0,0,7.8292,Q,Mr,0,0
1,893,3,female,47.0,1,0,7.0000,S,Mrs,1,0
2,894,2,male,62.0,0,0,9.6875,Q,Mr,0,0
3,895,3,male,27.0,0,0,8.6625,S,Mr,0,0
4,896,3,female,22.0,1,1,12.2875,S,Mrs,2,0


In [78]:
df_test['Survived'].value_counts()

Survived
0    265
1    153
Name: count, dtype: int64

In [80]:
submission = df_test[['PassengerId', 'Survived']]
submission.to_csv('submission.csv', index=False)